Install

In [2]:
!curl -o- -s -S -L https://raw.githubusercontent.com/bitcoinops/taproot-workshop/Colab/setup-colab-env.sh | bash

Colab environment setup.


In [3]:
import util
from test_framework.address import program_to_witness
from test_framework.key import generate_key_pair
from test_framework.messages import COutPoint, CTransaction, CTxIn, CTxOut, CTxInWitness
from test_framework.script import SegwitV0SignatureHash, SIGHASH_ALL, hash160, get_p2pkh_script
from util import TestWrapper
import tempfile
import os
from io import BytesIO

Source directory configured as /content/bitcoin/


Start 2 nodes

In [4]:
test = TestWrapper()
# Start TestNodes
test.num_nodes = 3
test.setup(num_nodes=3)
print("count nodes", test.num_nodes)

2020-04-23T16:39:36.854000Z TestFramework./tmp/bitcoin_func_test_kk2m6d8k (INFO): Initializing test directory /tmp/bitcoin_func_test_kk2m6d8k
count nodes 3


get balanses nodes and generate address

In [5]:
node1 = test.nodes[0] # coin sender
node2 = test.nodes[1] # coin receiver
node3 = test.nodes[2] # mainer
balance1 = node1.getbalance()
print('Balance node 1:', balance1)
balance2 = node2.getbalance()
print('Balance node 2:', balance2)
balance3 = node3.getbalance()
print('Balance node 3:', balance3)


Balance node 1: 0E-8
Balance node 2: 0E-8
Balance node 3: 0E-8


In [6]:
# Generate a new key pair
privkey, pubkey = generate_key_pair()
print("Pubkey: {}\n".format(pubkey.get_bytes().hex()))

# Get the hash160 of the public key for the witness program
program = hash160(pubkey.get_bytes())
print("Witness program: {}\n".format(program.hex()))

# Create (regtest) bech32 address
version = 0x00
reward_address_node1 = node1.getnewaddress(address_type="bech32") # generate reward address sender
print("bech32 reward address node 1: {}".format(reward_address_node1)) 

reward_address_node3 = node3.getnewaddress(address_type="bech32") # generate reward address miner
print("bech32 reward address node 3: {}".format(reward_address_node3)) 

address = node2.getnewaddress(address_type="bech32") # generate addres receiver
print("bech32 address node 2: {}".format(address))


Pubkey: 038b5eed55417be765943572e69454e6c996f61ed0217cffcdb4acad0c7e23f954

Witness program: e061658261b03a027fa7cd15a8511f0cb6ffeaef

bech32 reward address node 1: bcrt1qz4yhu0q6wkx4g66sl9je9zgtjwkv72zn46esq4
bech32 reward address node 3: bcrt1q8m2a9wamgwqygr43vfzeujc45pn7256hhdzxne
bech32 address node 2: bcrt1qqjyjw2umvq0nz9908nfe5394fe330wsynah6vh


In [8]:
    node1.generatetoaddress(2, reward_address_node1) # add node1 (sender) to 50 btc 
    index = 0
    while node1.getbalance() < 40:
      node3.generatetoaddress(101, reward_address_node3) # approve balance replenishment for node 1
      print(index)
      index +=1
      if index >= 100: 
        print ("failed, retry the request")
        break

    balance1 = node1.getbalance()
    print("Balance node 1: {}\n".format(balance1))

0
1
Balance node 1: 50.00000000



In [9]:
    unspent_txid1 = node1.listunspent(1)[-1]["txid"]
    inputs1 = [{"txid": unspent_txid1, "vout": 0}]

    # Create a raw transaction sending 1 BTC to the address, then sign and send it.
    # We won't create a change output, so maxfeerate must be set to 0
    # to allow any fee rate.
    tx_hex1 = node1.createrawtransaction(inputs=inputs1, outputs=[{address: 1}])

    res1 = node1.signrawtransactionwithwallet(hexstring=tx_hex1)

    tx_hex1 = res1["hex"]
 
    txid1 = node1.sendrawtransaction(hexstring=tx_hex1, maxfeerate=0)

    tx_hex1 = node1.getrawtransaction(txid1)

    # Reconstruct wallet transaction locally
    tx = CTransaction()
    tx.deserialize(BytesIO(bytes.fromhex(tx_hex1)))
    tx.rehash()

'd995ea8be41a41b8c525801494b2dcddc4dca9a7073b03938f40502a0be67bbf'

Transactions:

In [10]:

# Create a spending transaction
spending_tx = test.create_spending_transaction(tx.hash)
print("Spending transaction:\n{}\n".format(spending_tx))

# Sign the spending transaction and append the witness
sighash = SegwitV0SignatureHash(script=get_p2pkh_script(program),
                                txTo=spending_tx,
                                inIdx=0,
                                hashtype=SIGHASH_ALL,
                                amount=100_000_000)
sig = privkey.sign_ecdsa(sighash) + chr(SIGHASH_ALL).encode('latin-1')
spending_tx.wit.vtxinwit.append(CTxInWitness([sig, pubkey.get_bytes()]))

node3.generatetoaddress(101, reward_address_node3) # approve transactions

transactions1 = node1.listtransactions()
print('transactions node 1:', transactions1)
transactions2 = node2.listtransactions()
print('transactions node 2:', transactions2)

print("Success!")

Spending transaction:
CTransaction(nVersion=1 vin=[CTxIn(prevout=COutPoint(hash=d995ea8be41a41b8c525801494b2dcddc4dca9a7073b03938f40502a0be67bbf n=0) scriptSig= nSequence=0)] vout=[CTxOut(nValue=0.50000000 scriptPubKey=00144198cb0ccccf72d62553641e3fb499792e9b4676)] wit=CTxWitness() nLockTime=0)

transactions node 1: [{'address': 'bcrt1qz4yhu0q6wkx4g66sl9je9zgtjwkv72zn46esq4', 'category': 'orphan', 'amount': Decimal('50.00000000'), 'label': '', 'vout': 0, 'confirmations': 0, 'generated': True, 'trusted': False, 'txid': 'aa8448e7a4f0dc4ef3dc10100d33058f458c1994434b32f35eb39f0ea4458a5c', 'walletconflicts': [], 'time': 1587660011, 'timereceived': 1587660011, 'bip125-replaceable': 'unknown'}, {'address': 'bcrt1qz4yhu0q6wkx4g66sl9je9zgtjwkv72zn46esq4', 'category': 'orphan', 'amount': Decimal('50.00000000'), 'label': '', 'vout': 0, 'confirmations': 0, 'generated': True, 'trusted': False, 'txid': 'e4ee8beeeb7a39634ed1ae328191b6c3abd52dc40d4736de2996a4164cd2ead3', 'walletconflicts': [], 'time':

check balanses:

In [11]:
balance1 = node1.getbalance()
print('Balance node 1:', balance1)
balance2 = node2.getbalance()
print('Balance node 2:', balance2)

Balance node 1: 0E-8
Balance node 2: 1.00000000


stop nodes:

In [13]:
# Shutdown TestNodes
test.shutdown()

2020-04-23T16:44:04.106000Z TestFramework./tmp/bitcoin_func_test_kk2m6d8k (INFO): Stopping nodes
2020-04-23T16:44:04.365000Z TestFramework./tmp/bitcoin_func_test_kk2m6d8k (INFO): Cleaning up /tmp/bitcoin_func_test_kk2m6d8k on exit
2020-04-23T16:44:04.367000Z TestFramework./tmp/bitcoin_func_test_kk2m6d8k (INFO): Tests successful


In [0]:
!ps -ax | grep bitcoind

In [0]:
!kill 875